In [7]:
import pandas as pd
import matplotlib
import sklearn
import requests
import io
from bs4 import BeautifulSoup


In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text

In [9]:
print(wiki_html)


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"d78d1283-f9d8-449a-a9d8-277ffc77ce38","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-rela

### Extract table data from the HTML
----

Using BeautifulSoup to find the table headers from the table with css class __wikitable sortable__. Since this page only has one of these tables, it will reference only one table. 

From there extract the table headers, while stripping any trailing whitespace or new line chars.

Then for each of the table row, extract the table detail and match each column with the table headers extracted before. 

Since there can be multiple 



In [23]:
soup = BeautifulSoup(wiki_html)
table = soup.find("table", attrs={"class":"wikitable sortable"})

# The first tr contains the field names.
headings = [th.get_text().strip() for th in table.find("tr").find_all("th")]

datasets = []
for row in table.find_all("tr")[1:]:
    dataset = dict(zip(headings, (td.get_text().strip() for td in row.find_all("td"))))
    datasets.append(dataset)



In [24]:
print(datasets)

[{'Postal Code': 'M1A', 'Borough': 'Not assigned', 'Neighborhood': 'Not assigned'}, {'Postal Code': 'M2A', 'Borough': 'Not assigned', 'Neighborhood': 'Not assigned'}, {'Postal Code': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'Postal Code': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'Postal Code': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'Postal Code': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'Postal Code': 'M7A', 'Borough': 'Downtown Toronto', 'Neighborhood': "Queen's Park, Ontario Provincial Government"}, {'Postal Code': 'M8A', 'Borough': 'Not assigned', 'Neighborhood': 'Not assigned'}, {'Postal Code': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue, Humber Valley Village'}, {'Postal Code': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'Postal Code': 'M2B', 'Borough': 'Not assigned', 'Neighborhood': 'Not assign

In [31]:
df = pd.DataFrame(data = datasets, columns = ['Postal Code', 'Borough', 'Neighborhood'])
df

Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                          Neighborhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       Not assigned  
178  Mimico NW, The Queensway West, South of Bloor,...  
179                                       Not assigned  

[180 rows x 3 columns]

### Filter the entries where the Borough is Not Assigned

In [41]:
df1 = df[df['Borough'] != 'Not assigned']
df1

Postal Code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
..          ...               ...   
160         M8X         Etobicoke   
165         M4Y  Downtown Toronto   
168         M7Y      East Toronto   
169         M8Y         Etobicoke   
178         M8Z         Etobicoke   

                                          Neighborhood  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
5                     Lawrence Manor, Lawrence Heights  
6          Queen's Park, Ontario Provincial Government  
..                                                 ...  
160      The Kingsway, Montgomery Road, Old Mill North  
165                               Church and Wellesley  
168  Business reply mail Processing Centre, South C...  
169  Old Mill South, King's Mill Park, Sunnylea, Hu...  
178  Mimico NW, The Queensway West, South of Bloor,...  

[103 rows x 3 columns]

In [42]:
print(df1.shape)

(103, 3)
